In [86]:
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
model_name = "digit82/kobart-summarization"

bart_config = BartConfig().from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name, config=bart_config)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [87]:
# from pprint import pprint
print(f'''
    BOS : {tokenizer.bos_token},
    EOS : {tokenizer.eos_token},
    Special_tokens : {tokenizer.special_tokens_map},
    Tokenizer's max_model_input_sizes : {tokenizer.model_max_length}

    BART Config max token length: {bart_config.max_position_embeddings}
    BART Embedding Layer: {model.get_encoder().embed_tokens}, {model.get_encoder().embed_positions}
    tokenizer의 encoder_max_len은 {model.get_encoder().embed_positions.num_embeddings} 이하여야 한다.

    BART Decoder Layer: {model.get_decoder().embed_tokens}, {model.get_decoder().embed_positions}
    tokenizer의 decoder_max_len은 {model.get_decoder().embed_positions.num_embeddings} 이하여야 한다.

''')


    BOS : <s>,
    EOS : </s>,
    Special_tokens : {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'},
    Tokenizer's max_model_input_sizes : 1000000000000000019884624838656

    BART Config max token length: 1026
    BART Embedding Layer: Embedding(30000, 768, padding_idx=3), BartLearnedPositionalEmbedding(1028, 768)
    tokenizer의 encoder_max_len은 1028 이하여야 한다.

    BART Decoder Layer: Embedding(30000, 768, padding_idx=3), BartLearnedPositionalEmbedding(1028, 768)
    tokenizer의 decoder_max_len은 1028 이하여야 한다.




In [88]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/data/ephemeral/home/nlp-5/song/"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

In [93]:
current_time = get_current_time()
output_dir = f"./outputs/exp_base_{current_time}"
save_eval_log_steps = 400 # save_step은 eval, log step의 배수여야 한다. 같이 맞춰주는 것이 편하다.
print(current_time)

0725172301


In [ ]:
config_data = {
    "general": {
        "data_path": "./data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": output_dir # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 250,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#PhoneNumber#', '#Address#', 
        '#DateOfBirth#','#PassportNumber#','#SSN#','#CardNumber#','#CarNumber#','#Email#']
    },
    "training": {
        "seed": 42,
        "output_dir": output_dir,
        "overwrite_output_dir": False,

        "save_total_limit": 2,
        "load_best_model_at_end": True,
        "save_steps": save_eval_log_steps,

        "logging_dir": output_dir,
        "logging_steps": save_eval_log_steps,

        "num_train_epochs": 35,
        "per_device_train_batch_size": 128,
        "remove_unused_columns": True,
        "fp16": True,
        "dataloader_drop_last": False,
        "group_by_length": True,
        
        "gradient_checkpointing": True,
        "gradient_checkpointing_kwargs": {"use_reentrant": False},
        "gradient_accumulation_steps": 1,
        # "torch_empty_cache_steps": 10,
        "dataloader_num_workers": 8,

        "per_device_eval_batch_size": 48,
        "evaluation_strategy": 'steps',
        "eval_steps": save_eval_log_steps,
        
        "predict_with_generate": True,
        "generation_max_length": 250,
        
        # Callbacks
        "early_stopping_patience": 2,
        "early_stopping_threshold": 0.001,

        # Optimizer
        "learning_rate": 1e-5,
        "warmup_steps": 10,
        "weight_decay": 1e-3,

        "report_to": "all" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "skiersong", # 팀 실험 시 organization 이름
        "project": "nlp-5",
        "name": f"baseline_test1_{current_time}", # 개별 실험 이름
        # "group": "", # 유사한 실험들은 같은 그룹으로 설정
        "notes": "Baseline-test", # 실험에 대한 추가 설명
    },
    "inference": {
        "ckt_dir": os.path.join(output_dir, 'best'), # 파인튜닝이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": os.path.join(output_dir, f"submission_{current_time}.csv"), # 제출할 csv 파일 저장 경로
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 200,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [97]:
import os
import yaml
from pprint import pprint

config_path = os.path.join(
    project_dir,'src','configs',
    f"config_base_{current_time}.yaml" # config 파일 이름을 설정
)
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': './data/',
             'model_name': 'digit82/kobart-summarization',
             'output_dir': './outputs/exp_base_0725172301'},
 'inference': {'batch_size': 32,
               'ckt_dir': './outputs/exp_base_0725172301/best',
               'early_stopping': True,
               'generate_max_length': 200,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'],
               'result_path': './outputs/exp_base_0725172301/submission_0725172301.csv'},
 'tokenizer': {'bos_token': '<s>',
               'decoder_max_len': 250,
               'encoder_max_len': 512,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#Person4#',
                                  '#PhoneNumber#',
                                  '#Address#',